# Tema 3: Programacion Genetica

Este tema es el mas corto del curso durando solo 1 clase. La programacion genetica es muy similar a los algoritmos geneticos que vimos las clases pasadas, pero ahora trabajaremos con arboles. Esto significa que podemos crear programas y encontrar el programa que mejor resuelve algun problema.

Comentarios enviarlos a Alexandre Bergel (abergel@dcc.uchile.cl).

Integrantes del grupo:

*   Integrante 1
*   Integrante 2
*   Integrante 3
*   Integrante 4

In [1]:
# para operaciones matematicas
import numpy as np
# para crear copias de los arboles
from copy import deepcopy
# random
import random

# esta funcion dice si el argumento es una funcion o no
def is_function(f):
    return hasattr(f, "__call__")

# esto nos permite recorrer una lista de a pedazos
# ejemplo el input es [1,2,3,4,5,6,7,8]
# con n=2 esta funcion nos hara tener
# [(1,2), (3,4), (5,6), (7,8)]
def chunks(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

## Arboles

El codigo abajo define una pequeña libreria que permite manipular arboles. Hay funciones para evaluar el arbol, serializarlo, copiarlo y reemplazar el nodo por otro. Con evaluar nos referimos a por ejemplo el arbol (4 + 5), que es un nodo SUMA y 2 TERMINALes, la evaluacion da 9.

In [2]:
class Node:
    def __init__(self, function):
        # nos aseguramos que el nodo recibe una funcion
        assert is_function(function)
        self.operation = function
        # esto nos permite contar cuantos argumentos recibe la funcion
        self.num_arguments = function.__code__.co_argcount
        self.arguments = []
        
    # funcion para evaluar un nodo (calcular el resultado)
    def eval(self):
        # es importante chequear que los argumentos que nos dieron
        # coincidan con los argumentos que necesitamos
        assert len(self.arguments) == self.num_arguments
        # evaluamos los argumentos, y luego los pasamos como argumentos a
        # la operacion
        # NOTA: esto -> *[...] significa que separa los elementos de la lista
        # ejemplo si tenemos `lista = [1,2,3]`
        # print(lista) -> [1,2,3]
        # print(*lista) -> 1 2 3
        # esto se llama `unpacking`.
        # lo necesitamos porque nuestra funcion recibe N argumentos
        # no una lista de tamaño N.
        return self.operation(*[node.eval() for node in self.arguments])
    
    # hace una lista con todos los hijos
    def serialize(self):
        l = [self]
        for node in self.arguments:
            l.extend(node.serialize())
        return l
    
    # copia al nodo
    def copy(self):
        return deepcopy(self)
    
    # reemplaza el nodo por otro
    def replace(self, otherNode):
        # aqui estamos haciendo algo medio hacky.
        # la forma correcta seria tener una referencia al nodo padre y
        # reemplazarse a si mismo por otro.
        # por temas de mantener el codigo corto lo hacemos asi
        # pero no lo hagan en casa!
        assert isinstance(otherNode, Node)
        self.__class__ = otherNode.__class__
        self.__dict__ = otherNode.__dict__


# esta clase representa todos los nodos quetienen 2 argumentos
class BinaryNode(Node):
    num_args = 2
    def __init__(self, function, left, right):
        # revisamos que todo sea un nodo y agregamos a las lista de
        # argumentos
        assert isinstance(left, Node)
        assert isinstance(right, Node)
        super(BinaryNode, self).__init__(function)
        self.arguments.append(left)
        self.arguments.append(right)
        

class AddNode(BinaryNode):
    def __init__(self, left, right):
        def _add(x,y):
            return x + y
        super(AddNode, self).__init__(_add, left, right)
        
    # esta es la funcion que define como se mostrara el nodo
    # como es un nodo que REPResenta la suma, lo mostramos como suma
    def __repr__(self):
        return "({} + {})".format(*self.arguments)
        
    
class SubNode(BinaryNode):
    def __init__(self, left, right):
        def _sub(x,y):
            return x - y
        super(SubNode, self).__init__(_sub, left, right)
        
    def __repr__(self):
        return "({} - {})".format(*self.arguments)
    
    
class MaxNode(BinaryNode):
    def __init__(self, left, right):
        def _max(x,y):
            return max(x,y)
        super(MaxNode, self).__init__(_max, left, right)
        
    def __repr__(self):
        return "max({{{}, {}}})".format(*self.arguments)

class MultNode(BinaryNode):
    def __init__(self, left, right):
        def _mult(x,y):
            return x * y
        super(MultNode, self).__init__(_mult, left, right)
        
    def __repr__(self):
        return "({} * {})".format(*self.arguments)
    
    
class TerminalNode(Node):
    # Este nodo representa una hoja de arbol. Es el nodo terminal
    # por lo que no tiene argumentos
    num_args = 0
    def __init__(self, value):
        # igual tenemos que representarlo como una funcion, por como
        # diseñamos el programa. Pero aqui va a ser una funcion vacia
        def _nothind(): pass
        super(TerminalNode, self).__init__(_nothind)
        self.value = value
        
    def __repr__(self):
        return str(self.value)
    
    def eval(self):
        # la evaluacion de un nodo terminal es el valor que contiene
        return self.value

El codigo siguiente permite definir un arbol programaticamente, es decir, a mano pero en el computador:

In [3]:
# define la expression: 5 - 3
expr = SubNode(TerminalNode(5), TerminalNode(3))
print(expr)

(5 - 3)


In [4]:
# Se puede evaluar un arbol a mandar un mensaje "eval()" a este
expr.eval()

2

**EJERCICIO**: Define programaticamente un arbol que representa la expression: `max({(5 + 6), (2 - 3)})`

In [5]:
#                _           _ 
#     /\        | |         | |
#    /  \   _ __| |__   ___ | |
#   / /\ \ | '__| '_ \ / _ \| |
#  / ____ \| |  | |_) | (_) | |
# /_/    \_\_|  |_.__/ \___/|_|
#                              
# WRITE YOUR ANSWER HERE                             

**EJERCICIO**: Verifica que evaluar este arbol retorna `11`

In [6]:
#  ______          _                  _   __                        _           _ 
# |  ____|        | |                (_) /_/             /\        | |         | |
# | |____   ____ _| |_   _  __ _  ___ _  ___  _ __      /  \   _ __| |__   ___ | |
# |  __\ \ / / _` | | | | |/ _` |/ __| |/ _ \| '_ \    / /\ \ | '__| '_ \ / _ \| |
# | |___\ V / (_| | | |_| | (_| | (__| | (_) | | | |  / ____ \| |  | |_) | (_) | |
# |______\_/ \__,_|_|\__,_|\__,_|\___|_|\___/|_| |_| /_/    \_\_|  |_.__/ \___/|_|
#                                                                                 
# WRITE YOUR ANSWER HERE                                                                                 

## AST - programas y generador

A continuacion presentamos la clase `AST`, permite definir un arbol aleatorio dada un conjunto de funciones permitidas, un conjunto de terminales y una profundidad maxima.

En nuestro ejemplo este arbol puede generar arboles que representen expresiones matematicas con `+`, `-`, `*` y `max`. Son programas simples. Pero facilmente podemos extenderlo a soportar expresiones condicionales como `if`, y ciclos como `for` y `while`. Es decir, podemos generar programas completos de forma aleatoria.

In [7]:
# un AST es un arbol que representa un programa, la idea aqui es tener 
# un generador, que pueda generar ASTs aleatorios
class AST:
    def __init__(self, allowed_functions, allowed_terminals, prob_terminal=0.3):
        # las funciones (nodos en nuestro caso) que nuestro programa puede tener
        self.functions = allowed_functions
        # los terminales admitidos en nuestro programa. Numeros por ejemplo
        self.terminals = allowed_terminals
        # para no tener un arbol infinitamente profundo, existe una posibilidad
        # de que, a pesar de que ahora toque hacer otro sub arbol, que se ignore
        # eso y se ponga un terminal en su lugar.
        self.prob = prob_terminal

    # esta funcion ya la hemos visto, nos permite llamar al AST como si fuera
    # una funcion. max_depth es la produndidad que queremos tenga el arbol
    def __call__(self, max_depth=10):
        # aqui tenemos una funcion auxiliar. Nos permitira hacer esto recursivo
        def create_rec_tree(depth):
            # si `depth` es mayor a 0, nos toca crear un sub-arbol
            if depth > 0:
                # elegimos una funcion aleatoriamente
                node_cls = random.choice(self.functions)
                # aqui iremos dejando los argumentos que necesita la funcion
                arguments = []
                # para cada argumento que la funcion necesite...
                for _ in range(node_cls.num_args):
                    # existe un `prob` probabilidad de que no sigamos creando
                    # sub-arboles y lleguemos y cortemos aqui para hacer
                    # un nodo terminal
                    if random.random() < self.prob:
                        arguments.append(create_rec_tree(0))
                    else:
                        # la otra es seguir creando sub-arboles recursivamente
                        arguments.append(create_rec_tree(depth - 1))
                
                # `arguments` es una lista y los nodos necesitan argumentos
                # asi que hacemos "unpacking" de la lista
                return node_cls(*arguments)
            else:
                # si `depth` es 0 entonces creamos un nodo terminal con
                # alguno de los terminales permitidos que definimos inicialmente
                return TerminalNode(random.choice(self.terminals))

        # llamamos a la funcion auxiliar para crear un arbol de profundidad `max_depth`
        return create_rec_tree(max_depth)

La clase `AST` permite definir un generador de arboles. Para definir este generador, se requiere un conjunto de funciones (nodos no terminales), un conjunto de valores (nodos terminales). Adicionalmente agregamos un argumento que permite saltarse la restriccion de profundidad del arbol. Es decir, si le decimos al generador que cree un arbol de produndidad 10, tendremos muchisimos nodos y funciones y nuestro arbol crecera demasiado. Para mitigar un poco esto, tenemos el argumento `prob_terminal`, que dada una probabilidad, ignora que le falten sub-arboles por generar y directamente retorna un terminal. Asi termina la generacion de sub-arboles por ese lado.

In [8]:
generator = AST([AddNode, SubNode, MaxNode, MultNode], list(range(10)), prob_terminal=0.4)

In [9]:
# genera un arbol de profundidad 2
tree1 = generator(2)
tree1

((6 * 2) + (7 - 0))

In [10]:
# "Serializar" permite obtener una lista de _todos_ los nodos que componen un arbol 
print(tree1.serialize())

[((6 * 2) + (7 - 0)), (6 * 2), 6, 2, (7 - 0), 7, 0]


En este caso tenemos 3 nodos. El nodo con la operacion y sus 2 argumentos, tambien nodos.

**EJERCICIO**: Construye un generador de arboles con una profondidad maxima de 10 y verifica que un valor `prob_terminal` alto permite generar arboles con menor profundidad.

In [11]:
#  _____            __                  _ _     _           _ 
# |  __ \          / _|                | (_)   | |         | |
# | |__) | __ ___ | |_ _   _  _ __   __| |_  __| | __ _  __| |
# |  ___/ '__/ _ \|  _| | | || '_ \ / _` | |/ _` |/ _` |/ _` |
# | |   | | | (_) | | | |_| || | | | (_| | | (_| | (_| | (_| |
# |_|   |_|  \___/|_|  \__,_||_| |_|\__,_|_|\__,_|\__,_|\__,_|
#                                                           
# WRITE YOUR ANSWER HERE                                                           

## Algoritmo de programacion genetica (GP)

La clase `GP` define el algoritmo completo. Es muy similar a `GA` que hemos visto, pero las operaciones geneticas cambiaron.

In [12]:
class GP:
    def __init__(self, function_set, terminal_set, fitness, pop_size=500, depth=3, crossover_rate=0.5,
                 mutation_rate=0.01, iterations=50, min_fitness=None, tournament_sampling=20):
        self.generator = AST(function_set, terminal_set)
        self.fitness_function = fitness
        self.population_size = pop_size
        # la probabilidad de hacer crossover entre 2 individuos
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.max_iterations = iterations
        if min_fitness is None:
            self.min_fitness = float("inf")
        else:
            self.min_fitness = min_fitness
        # cuantos invididuos random se seleccionan por torneo
        self.tournament_sampling = tournament_sampling
        # la profundidad de los arboles
        self.depth = depth

    def run(self):
        iters = 0
        best_fitness_list = []
        avg_list = []
        # creamos la poblacion
        population = self.generate_population()
        # evaluamos la poblacion
        fitness = self.get_fitness(population)
        # el mejor individuo sera una tupla (individuo, fitness)
        best_individual = (population[np.argmax(fitness)], max(fitness))

        # mientras no hayamos terminado las iteraciones Y nuestro mejor
        # individuo sea menor que una cota que pongamos -> seguimos
        while iters < self.max_iterations and best_individual[1] < self.min_fitness:
            print(f"iteration {iters} of {self.max_iterations}")

            # agregamos el fitness del mejor individuo
            best_fitness_list.append(best_individual[1])
            # y metemos el promedio a la lista de los promedios
            avg_list.append(np.mean(fitness))

            print(f"best is: {best_individual[0]}\nwith fitness {best_individual[1]}")
            print("-" * 20)
            
            # hacemos la seleccion basandonos en los fitness de la poblacion
            parents = self.select(population, fitness)
            # creamos una nueva poblacion
            population = self.create_new_population(parents)
            # calculamos el nuevo fitness para esta poblacion
            fitness = self.get_fitness(population)
            # hacemos de nuevo al mejor individuo
            best_individual = (population[np.argmax(fitness)], max(fitness))

            iters += 1
            
        print(f"iteration {iters} of {self.max_iterations}")

        best_fitness_list.append(max(fitness))
        avg_list.append(np.mean(fitness))

        print(f"best is: {best_individual[0]}\nwith fitness {best_individual[1]}")
        print("-" * 20)
        return best_fitness_list, avg_list, best_individual[0]

    # creamos la nueva poblacion
    def generate_population(self):
        population = []
        for _ in range(self.population_size):
            # generamos un nuevo arbol de profundidad `depth` aleatorio
            population.append(self.generator(self.depth))
        return population

    # calculamos el fitness de cada individuo
    def get_fitness(self, population):
        fitness = []
        for p in population:
            # fitness de cada individuo
            fitness.append(self.fitness_function(p))
        return fitness

    # seleccionamos a los padres
    def select(self, population, pop_fitness):
        parents = []
        # tenemos que hacer 2N torneos para una poblacion de N elementos
        for _ in range(2 * self.population_size):
            # elejimos `tournament_sampling` elementos aleatorios
            competitors_idx = np.random.choice(len(population), self.tournament_sampling)
            # hacemos un mapeo de (individuo, fitness individuo)
            competitors = [(population[idx], pop_fitness[idx]) for idx in competitors_idx]

            # elegimos al mejor de esos aleatorios que elegimos
            # como creamos llegan (individuo, fitness individuo)
            # la eleccion ocurre basandonos en el fitness (seleccionamos [1])
            winner = max(competitors, key=lambda x: x[1])
            # el individuo esta en [0]
            parents.append(winner[0])
        return parents

    # usando los padres, creamos nueva poblacion
    def create_new_population(self, parents):
        new_population = []
        # usamos la funcion `chunks` para recorrer de a 2 elementos
        for p1, p2 in chunks(parents, 2):
            # creamos un nuevo elemento
            new_population.append(self.create_new_element(p1, p2))
        return new_population

    # crea un nuevo elemento basandonos en los 2 padres
    def create_new_element(self, p1, p2):
        # primero hay que decidir si hay crossover o no
        rand = random.random()
        # generalmente es bien probable que lo hagamos
        # pero existe la probabilidad de que no
        if rand <= self.crossover_rate:
            # hacemos una copia del padre 1
            new_element = p1.copy()
            # elegimos un punto del padre 1 a cortar aleatoriamente
            point_parent_1 = random.choice(new_element.serialize())
            # elegimos un punto en el padre 2 a cortar, tambien aleatoriamente
            # y creamos una copia de ese subarbol
            point_parent_2 = random.choice(p2.serialize()).copy()
            # reemplzamos el punto del padre 1 que seleccionamos, con
            # el punto que elegimos del padre 2
            point_parent_1.replace(point_parent_2)
        else:
            # si no hacemos crossover, entonces elegimos al mejor de los 2
            # padres. Necesitamos la copia.
            if self.fitness_function(p1) >= self.fitness_function(p2):
                new_element = p1.copy()
            else:
                new_element = p2.copy()
                
        # aqui hacemos la putacion
        rand = random.random()
        if rand <= self.mutation_rate:
            # si toca hacer mutacion, creamos un nuevo arbol
            # de profundidad aleatoria entre 0 y `depth.
            depth = random.randint(0, self.depth)
            # reemplazamos el arbol que recien creamos en algun punto aleatorio
            # del nuevo individuo que sacamos del crossover.
            random.choice(new_element.serialize()).replace(self.generator(depth))

        return new_element

### Algoritmo en accion

El codigo siguiente usa `GP` para buscar una expression que se acerca al valor `target_number`. El desafio es que dado un conjunto de numeros, como podemos operarlos tal que la evaluacion del arbol resultate de un numero muy cercano a `target_number`.

In [13]:
# busquemos este numero
target_number = 46436

# el 0 es el mejor! Todo lo demas es negativo.

def fitness_fn(element):
    # esta fitness evalua el elemento sacando que tan lejos
    # esta del numero que queremos llegar
    generated = element.eval()
    # como queremos maximizar, es negativo
    return -abs(target_number - generated)

def length_fitness_fn(element):
    # esta hace lo mismo que antes, pero pone una penalizacion
    # cuando el tamaño del arbol es muy grande
    return fitness_fn(element) - len(element.serialize())

# va cambiando el fitness cuando se queda pegado por muchas iteraciones
# cambia fitness entre 
# -> encuentra lo mejor que puedas, aumentar el tamaño con `fitness_fn`
# -> reduce el tamaño con `length_fitness_fn` pero intenta mantener el fitness
class MetaFitness():
    def __init__(self, patience=200, fitness_switch=50):
        self.best = float("inf")
        # cuanto estamos dispuestos a esperar
        self.patience = patience
        # si llegamos a este valor, cambiamos de fitness
        self.fitness_switch = fitness_switch

        # cuantas veces llevamos sin vambios
        self.current_wait = 0
        # vamos a ir variando esta
        self.current_f = fitness_fn

        # variables auxiliares para que nos de el estado
        # esta significa que usaremos `fitness_fn`
        self.use_standard = True
        # esta significa que aun no hemos roto la paciencia
        self.fast_approx = True

    def __call__(self, element):
        # si aun no rompemos la paciencia pero encontramos un individuo bueno
        if self.fast_approx and self.best > self.fitness_switch:
            # cambiamos a minimizar el tamaño del arbol
            self.current_f = length_fitness_fn
            self.use_standard = False
            self.fast_approx = False

        # si ya rompimos la paciencia
        if self.current_wait >= self.patience:
            print("Changing fitness")
            # cambiamos de fitness
            self.use_standard = not self.use_standard
            # reiniciamos la paciencia
            self.current_wait = 0

        # si ya rompimos la paciencia, vamos variando entre las 2 fitness
        if not self.fast_approx:
            # vamos variando entre ambas cada vez que rompamos la paciencia
            if self.use_standard:
                self.current_f = fitness_fn
            else:
                self.current_f = length_fitness_fn

        # vamos guardando el mejor fitness. Si es el mejor dejamos de esperar
        # y lo guardamos.
        # si no es mejor que el que teniamos guardado, agregamos 1 
        # a la espera
        e_fitness = self.current_f(element)
        if e_fitness > self.best:
            self.best = e_fitness
            self.current_wait = 0
        else:
            self.current_wait += 1

        return e_fitness
    

Ejecuta el siguiente codigo y ve como le va al algoritmo con la funcion de fitness que solo considera que tan cerca esta de al solucion. Funciona?
Que tal si usas las otras fitness?

In [14]:
_pop_size = 1000
_iterations = 200
ga = GP(function_set=[AddNode, SubNode, MaxNode, MultNode], 
        terminal_set=[5,76,8,32,6,26,5], 
        fitness=fitness_fn, 
        #fitness=length_fitness_fn,
        #fitness=MetaFitness(patience=_pop_size*10), 
        pop_size=_pop_size, 
        depth=5, 
        crossover_rate=0.9,
        mutation_rate=0.1, 
        iterations=_iterations, 
        min_fitness=0, 
        tournament_sampling=_pop_size//10)
    
best_fitness_list, avg_list, best_individual = ga.run()

iteration 0 of 200
best is: max({5, max({(8 * ((76 - 5) * (76 + 5))), ((5 + 5) * 32)})})
with fitness -428
--------------------
iteration 1 of 200
best is: (((32 * (26 + 8)) + max({76, (6 * 5)})) * (5 * 8))
with fitness -124
--------------------
iteration 2 of 200
best is: max({76, ((5 * 76) + (max({max({32, 32}), (76 * 6)}) * ((5 * 5) + max({76, 5}))))})
with fitness 0
--------------------


In [15]:
print(best_individual)
print(f"Tree length: {len(best_individual.serialize())}")
print(f"best individual: {best_individual.eval()} vs {target_number}")

max({76, ((5 * 76) + (max({max({32, 32}), (76 * 6)}) * ((5 * 5) + max({76, 5}))))})
Tree length: 21
best individual: 46436 vs 46436


## Ejercicios

**EJERCICIO**: Usando `GP` encuentra una expresion que se acerque al numero `459`, usando solo `suma`, `resta` y `maximo` con los valores 25, 7, 8, 100, 4 y 2.

In [16]:
#  _  _   _____ ___  
# | || | | ____/ _ \ 
# | || |_| |__| (_) |
# |__   _|___ \\__, |
#    | |  ___) | / / 
#    |_| |____/ /_/  
#                    
# WRITE YOUR ANSWER HERE                     

**EJERCICIO**: Construye una hotmap con la variación de la tasa de mutacion y el tamaño de la populación. Cual son sus conclusiones?

Ayuda: es lo mismo que hicimos con `GA`.

In [17]:
#  _    _       _                         
# | |  | |     | |                        
# | |__| | ___ | |_ _ __ ___   __ _ _ __  
# |  __  |/ _ \| __| '_ ` _ \ / _` | '_ \ 
# | |  | | (_) | |_| | | | | | (_| | |_) |
# |_|  |_|\___/ \__|_| |_| |_|\__,_| .__/ 
#                                  | |    
#                                  |_|    
#
# WRITE YOUR ANSWER HERE

**EJERCICIO**: Agrega la funcion de division al conjunto de funciones. Esta hay que implementarla arriba donde estan las otras funciones, pero hay un detalle, **cuidado con dividir por 0**!

In [18]:
#  _____  _       _     _             
# |  __ \(_)     (_)   (_)            
# | |  | |___   ___ ___ _  ___  _ __  
# | |  | | \ \ / / / __| |/ _ \| '_ \ 
# | |__| | |\ V /| \__ \ | (_) | | | |
# |_____/|_| \_/ |_|___/_|\___/|_| |_|
#                                     
# WRITE YOUR ANSWER HERE                                     

**EJERCICIO**: En la implementacion actual tenemos valores numericas como nodos terminales. Sera posible agregar variables? Algo como, por ejemplo, `x`. Es decir, una variable que por el momento no tiene valor especifico, pero que podemos reemplazarla por un valor despues. 

1. Que impacto tendria respecto a la evaluación del arbol?
2. Podriamos usar esta variable para ahora encontrar una expresion que se acerque, por ejemplo, a $x^2 + 10x + 5$?
3. Comenta como implementarias esto, que cosas tendrias que cambiar y que cosas agregar.

**Aqui tu respuesta**

**EJERCICIO**: Implementa el codigo para la solucion propuesta en el ejercicio anterior.

In [19]:
# __      __        _       _     _           
# \ \    / /       (_)     | |   | |          
#  \ \  / /_ _ _ __ _  __ _| |__ | | ___  ___ 
#   \ \/ / _` | '__| |/ _` | '_ \| |/ _ \/ __|
#    \  / (_| | |  | | (_| | |_) | |  __/\__ \
#     \/ \__,_|_|  |_|\__,_|_.__/|_|\___||___/
#                                             
# WRITE YOUR ANSWER HERE (optional exercise)                                             